# Geração de Databases com SQLalchemy e SQLite

O presente notebook demonstra a construção de bancos de dados (BD) SQL locais a partir do uso das ferramentas SQLite e SQLalchemy, para fins de realização de uma análise exploratória de dados descritos em arquivos CSV. Para tal, fizemos uso do dataset [Covid-19 Data Sharing](https://agencia.fapesp.br/covid-19-data-sharingbr-makes-more-datasets-available/35348) disponibilizado pela Agência FAPESP.

## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

## Aviso

O processo de população dos bancos de dados é **computacionalmente intensivo**, dado o volume de dados e as verificações colocadas para sua correta formatação. O processamento do dataset do Grupo Fleury, o maior dentre todos, necessitou-me o uso do seviço online [Google Colab](https://colab.research.google.com), o qual disponibiliza placas de CPU com elevada memória RAM (Observou-se um pico de consumo de 36 GiB desta), para a execução remota deste notebook.

## Setup

Carregamento das dependências deste projeto

In [6]:
import enum
import pandas as pd
import re
from sqlalchemy import (
    CheckConstraint as constraint,
    Enum,
    ForeignKey as fk,
    PrimaryKeyConstraint as pkc,
    String,
    create_engine,
)
from sqlalchemy.orm import (
    declarative_base,
    Mapped,
    relationship,
    sessionmaker,
    mapped_column as column,
    validates,
)
from pathlib import Path
from sqlalchemy.ext.hybrid import hybrid_property
from tqdm import tqdm
from typing import Any, Annotated, final

## Funções e tipos de dados auxiliares

Algumas funções e tipos de dados os quais utilizamos em nossa implementação, mas cuja funcionalidade provavelmente não é crucial ao caso geral.

In [7]:
def backref(back_populates: str) -> Mapped[Any]:
    return relationship(back_populates=back_populates)


def childOf(back_populates: str) -> Mapped[Any]:
    return relationship(
        back_populates=back_populates,
        cascade="all, delete-orphan",
    )


def get_project_dir():
    """A function to get the project directory that works in both python scripts
    and jupyter notebooks"""

    try:
        # Try the script approach first
        return Path(__file__).parent.resolve()
    except NameError:
        # Fall back to current directory for notebooks
        return Path.cwd()

def normalize_column_name(column_name: str) -> str:
    """
    Convert column name from UU_UUU_UUU format to UU_Uuu_Uuu format
    Example: "ID_PACIENTE" -> "ID_Paciente", "IC_SEXO" -> "IC_Sexo"
    """
    match column_name:
        case "CD_UF":
            return column_name
        case "CD_CEPREDUZIDO":
            return "CD_CEPReduzido"
        case _:
            parts = column_name.split("_")
            return "_".join(
                [parts[0].upper()] + [part.capitalize() for part in parts[1:]]
            )


@final
class EstadoBrasil(enum.Enum):
    AC = "AC"  # Acre
    AL = "AL"  # Alagoas
    AP = "AP"  # Amapá
    AM = "AM"  # Amazonas
    BA = "BA"  # Bahia
    CE = "CE"  # Ceará
    DF = "DF"  # Distrito Federal
    ES = "ES"  # Espírito Santo
    GO = "GO"  # Goiás
    MA = "MA"  # Maranhão
    MT = "MT"  # Mato Grosso
    MS = "MS"  # Mato Grosso do Sul
    MG = "MG"  # Minas Gerais
    PA = "PA"  # Pará
    PB = "PB"  # Paraíba
    PR = "PR"  # Paraná
    PE = "PE"  # Pernambuco
    PI = "PI"  # Piauí
    RJ = "RJ"  # Rio de Janeiro
    RN = "RN"  # Rio Grande do Norte
    RS = "RS"  # Rio Grande do Sul
    RO = "RO"  # Rondônia
    RR = "RR"  # Roraima
    SC = "SC"  # Santa Catarina
    SP = "SP"  # São Paulo
    SE = "SE"  # Sergipe
    TO = "TO"  # Tocantins


@final
class Origem(enum.Enum):
    LAB = "LAB"  # Unidade laboratorial
    HOSP = "HOSP"  # Unidade hospitalar
    UTI = "UTI"  # Unidade de Tratamento Intensivo


Date = Annotated[str, column(String(10))]

## Definição das tabelas comuns

Abaixo descrevemos a estrutura pretendida às tabelas Pacientes, ExamLabs e Despachos, comuns a todos o BDs.

In [17]:
Base = declarative_base()

class PacienteBase(Base):
    __abstract__: bool = True

    IC_Sexo: Mapped[str] = column(
        String(1), constraint("IC_Sexo IN ('M', 'F')", name="check_sexo")
    )
    AA_Nascimento: Mapped[str] = column(String(4))
    CD_Pais: Mapped[str | None] = column(String(2))
    CD_UF: Mapped[str | None] = column(Enum(EstadoBrasil))
    CD_Municipio: Mapped[str | None]
    CD_CEPReduzido: Mapped[str | None]

    @validates("AA_Nascimento")
    def validates_nascimento(self, _key: str, value: str) -> str:
        match value:
            case "AAAA" | "YYYY":
                return value
            case year if len(year) == 4 and year.isdigit():
                return year
            case _:
                raise ValueError(
                    f"Invalid AA_Nascimento value: {value}. Must be 'AAAA', 'YYYY', or a 4-digit number"
                )

    @validates("CD_Pais")
    def validate_pais(self, _key: str, value: str) -> str | None:
        return value if value != "XX" else None


    @validates("CD_UF")
    def validate_uf(self, _key: str, value: str) -> str | None:
        return value if value != "UU" else None

    @validates("CD_Municipio")
    def validate_municipio(self, _key: str, value: str) -> str | None:
        return value if value != "MMMM" else None

    @validates("CD_CEPReduzido")
    def validate_cepreduzido(self, _key: str, value: str) -> str | None:
        return value if value != "CCCC" else None


class Paciente(PacienteBase):
    """
    Tabela de pacientes Covid-19 FAPESP

    Atributos:
        ID_Paciente: Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece). 32 caracteres alfanuméricos
        IC_Sexo: Sexo do Paciente. 1 caracter alfanumérico. F - Feminino; M - Masculino
        AA_Nascimento: Ano de nascimento do Paciente. 4 caracteres alfanuméricos.
            Os 4 dígitos do ano do nascimento; ou
            AAAA - para ano de nascimento igual ou anterior a 1930 (visando anonimização);
            YYYY - quaisquer outros anos, em caso de anonimização do ano
        CD_Pais: Pais de residencia do Paciente. 2 caracteres alfanuméricos. BR ou XX (país estrangeiro)
        CD_UF: Unidade da Federacao de residencia do Paciente. 2
        caracteres alfanuméricos
        CD_Municipio: Municipio de residencia do Paciente. Alfanumérico.
            Nome do município por extenso,
            ou MMMM - quando houver necessidade de anonimização ou estrangeiro
        CD_CEPReduzido: [Descrição não encontrada nos comentários]
    """

    __tablename__: str = "Pacientes"

    ID_Paciente: Mapped[str] = column(String(32), primary_key=True)

    # Relações
    exames: Mapped[list["ExamLab"]] = childOf("paciente")
    desfechos: Mapped[list["Desfecho"]] = childOf("paciente")



class ExamLab(Base):
    """
    Tabela de exames Covid-19 FAPESP

    Atributos:
        ID_Paciente: Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece). 32 caracteres alfanuméricos
        ID_Atendimento: Identificação única do atendimento. Correlaciona com o ID_ATENDIMENTO de todas as tabelas onde aparece. 32 caracteres alfanuméricos
        DT_Coleta: Data em que o material foi coletado do paciente (DD/MM/AAAA)
        DE_Origem: Local de Coleta do exame. 4 caracteres alfanuméricos:
            LAB – Exame realizado por paciente em uma unidade de atendimento laboratorial;
            HOSP – Exame realizado por paciente dentro de uma Unidade Hospitalar;
            UTI - exame realizado na UTI
        DE_Exame: Descrição do exame realizado. Alfanumérico.
            Exemplo: HEMOGRAMA, sangue total / GLICOSE, plasma / SODIO, soro / POTASSIO, soro.
            Um exame é composto por 1 ou mais analitos.
        DE_Analito: Descrição do analito. Alfanumérico. Exemplo: Eritrócitos / Leucócitos / Glicose / Ureia / Creatinina.
            Para o exame Hemograma, tem o resultado de vários analitos: Eritrócitos, Hemoglobina, Leucócitos, Linfócitos, etc.
            A maioria dos exames tem somente 1 analito, por exemplo Glicose, Colesterol Total, Uréia e Creatinina.
        DE_Resultado: Resultado do exame, associado ao DE_ANALITO. Alfanumérico. Se DE_ANALITO exige valor numérico, NNNN se inteiro ou NNNN,NNN se casas decimais;
            Se DE_ANALITO exige qualitativo, String com domínio restrito;
            Se DE_ANALITO por observação microscópica, String conteúdo livre.
            Exemplo de dominio restrito - Positivo, Detectado, Reagente, nâo reagente, etc.
            Exemplo de conteúdo livre - 'não foram observados caracteres tóxico-degenerativos nos neutrófilos, não foram observadas atipias linfocitárias'
        CD_Unidade: Unidade de Medida utilizada na Metodologia do laboratório específico para analisar o exame. Alfanumérico.
            Exemplo: g/dL (gramas por decilitro)
        CD_ValorReferencia: Faixa de valores de referência. Alfanumérico. Resultado ou faixa de resultados considerado normal para este analito.
            Exemplo para Glicose: 75 a 99
    """

    __tablename__: str = "ExamLabs"

    index: Mapped[int] = column(primary_key=True)
    ID_Paciente: Mapped[str] = column(fk("Pacientes.ID_Paciente"))
    ID_Atendimento: Mapped[str | None] = column(String(32))
    DE_Exame: Mapped[str | None]
    DE_Resultado: Mapped[str | None]
    DT_Coleta: Mapped[Date | None]
    DE_Origem: Mapped[str | None] = column(Enum(Origem))
    DE_Analito: Mapped[str | None]
    CD_Unidade: Mapped[str | None]
    DE_Valor_Referencia: Mapped[str | None]

    @validates("DE_Origem")
    def validate_de_origem(self, _key: str, value: str | None) -> str | None:
        # Convert pandas NaN to None
        if pd.isna(value) or value == '':
            return None
        return value

    @hybrid_property
    def DE_resultNum(self) -> float | None:
        """
        Extrai valor numérico do resultado ou atribui códigos especiais para resultados textuais.
        Baseado na lógica do script COVID19_Corrige_21_02.sql
        """
        if not self.DE_Resultado:
            return None

        # Extrai valor numérico do resultado
        numeric_match = re.search(r"-?\d+[,.]?\d*", self.DE_Resultado)
        if numeric_match:
            numeric_str = numeric_match.group().replace(",", ".")
            try:
                return float(numeric_str)
            except ValueError:
                pass

        # Aplica códigos especiais para exames COVID
        if self.DE_Exame and re.search(
            r"(covid)|(sars.cov.2)|(corona)", self.DE_Exame, re.IGNORECASE
        ):
            resultado_lower = self.DE_Resultado.lower()

            if re.search(r"detectados anticorpos", resultado_lower):
                return -1000.0
            elif re.search(
                r"(n.o detectado)|(n.o reagente)|(negativo)|(aus.ncia de anticorpos)",
                resultado_lower,
            ):
                return -1111.0
            elif re.search(r"(detectado)|(reagente)|(positivo)", resultado_lower):
                return -1000.0
            elif re.search(r"(indetect.avel)|(inconclusivo)", resultado_lower):
                return -1234.0
            else:
                return -2222.0

        return None

    # Relações
    paciente: Mapped["Paciente"] = backref("exames")


class Desfecho(Base):
    """
    Tabela de desfechos Covid-19 FAPESP

    Atributos:
        ID_Paciente: Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece. 32 caracteres alfanuméricos)
        ID_Atendimento: Identificação única do atendimento. Cada atendimento tem um desfecho. Correlaciona com ID_ATENDIMENTO de todas as tabelas onde aparece
        DT_Atendimento: Data de realização do atendimento - (DD/MM/AAAA)
        DE_TipoAtendimento: Descrição do tipo de atendimento realizado. Texto livre. Exemplo: Pronto atendimento.
        ID_Clinica: Identificação da clínica onde o evento aconteceu. Numérico. Exemplo: 1013
        DE_Clinica: Descrição da clínica onde o evento aconteceu. Texto livre. Exemplo: Cardiologia
        DT_Desfecho: Data do desfecho - (DD/MM/YYYY) ou string DDMMAA se DE_DESFECHO for óbito
        DE_Desfecho: Descriçao do desfecho. Texto livre. Exemplo: Alta médica melhorado
    """

    __tablename__: str = "Desfechos"

    ID_Paciente: Mapped[str] = column(fk("Pacientes.ID_Paciente"))
    ID_Atendimento: Mapped[str] = column(String(32))
    DT_Atendimento: Mapped[Date]
    DE_Tipo_Atendimento: Mapped[str]
    ID_Clinica: Mapped[int]
    DE_Clinica: Mapped[str]
    DT_Desfecho: Mapped[Date | None]
    DE_Desfecho: Mapped[str]

    @validates("DT_Desfecho")
    def validate_dt_desfecho(self, _key: str, value: str) -> str | None:
        return value if value != "DDMMAA" else None

    # Relações
    paciente: Mapped["Paciente"] = backref("desfechos")

    __table_args__: tuple[pkc,] = (pkc("ID_Paciente", "ID_Atendimento"),)


## População dos bancos de dados

Abaixo descrevemos a lógica para população dos bancos de dados. Os datasets que descrevem a cada tabela de cada banco de dados estão localizados em uma pasta `dataset` colocada no mesmo diretório que este notebook.

```bash
~/Public/USP/Ciência da Computação/Semestre 6/Mineração de dados/01-Introdução-Preparação de dados main*
❄️impure ❯ exa --tree
.
├── datasets
│   ├── BPSP
│   │   ├── BPSP_Desfechos.csv
│   │   ├── BPSP_ExamLabs.csv
│   │   └── BPSP_Pacientes.csv
│   ├── Einstein
│   │   ├── Einstein_ExamLabs.csv
│   │   └── Einstein_Pacientes.csv
│   ├── GrupoFleury
│   │   ├── GrupoFleury_ExamLabs.csv
│   │   └── GrupoFleury_Pacientes.csv
│   ├── HC
│   │   ├── HC_ExamLabs.csv
│   │   └── HC_Pacientes.csv
│   └── HSL
│       ├── HSL_Desfechos.csv
│       ├── HSL_ExamLabs.csv
│       └── HSL_Pacientes.csv
└── 'Geração de Databases com SQLalchemy e SQLite.ipynb'
```
Como se vê, os datasets encontram-se nomeados de maneira padronizada, e os nomes das colunas de suas tabelas correspondem aos nomes dados aos atributos das classes que aqui descrevem as tabelas contidas nos BDs.

In [9]:
cwd = get_project_dir()
hospitals = ["BPSP", "Einstein", "GrupoFleury", "HC", "HSL"]

In [ ]:
for hospital in hospitals:
    engine = create_engine(f"sqlite:///{cwd}/../fapcov2103/{hospital}.db")
    Session = sessionmaker(bind=engine)
    session = Session()

    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)
    session.commit()

    pbar = tqdm([("Pacientes", False), ("ExamLabs", True), ("Desfechos", False)])
    for table, indexed in pbar:
        try:
            df = pd.read_csv(
                f"{cwd}/datasets/{hospital}/{hospital}_{table}.csv",
                delimiter="|",
                encoding="utf-8",
                low_memory=False,
            )
            df.rename(columns=normalize_column_name, inplace=True)
            pbar.set_description(
                desc=f"Populating database {hospital}, adding to table {table}"
            )
            match table:
                case "Pacientes":
                    session.add_all(
                        [Paciente(**row.to_dict()) for _, row in df.iterrows()]
                    )
                case "ExamLabs":
                    session.add_all(
                        [ExamLab(**row.to_dict()) for _, row in df.iterrows()]
                    )
                case _:
                    session.add_all(
                        [Desfecho(**row.to_dict()) for _, row in df.iterrows()]
                    )

        except FileNotFoundError:
            pass
    session.commit()
    session.close()

O resultado esperado desta execução é a criação dos seguintes BDs:
```bash
~/Public/USP/Ciência da Computação/Semestre 6/Mineração de dados/01-Introdução-Preparação de dados main*
❯ exa --tree -l ../fapcov2103/
drwxr-xr-x    - user 23 Aug 11:01 ../fapcov2103
.rw-r--r-- 973M user 23 Aug 00:09 ├── BPSP.db
.rw-r--r--  19M user 23 Aug 11:01 ├── D2.db
.rw-r--r-- 432M user 23 Aug 00:30 ├── Einstein.db
.rw-r--r-- 1,6G user 23 Aug 10:38 ├── GrupoFleury.db
.rw-r--r-- 400M user 23 Aug 07:59 ├── HC.db
.rw-r--r-- 250M user 23 Aug 08:19 └── HSL.db
```
Estruturados tal qual exibe os seguinte diagrama gerado usando a ferramenta DBeaver:

![Estrutura do BD, onde Pacientes figura como uma tabela associada a ExamLabs e Despachos](imgs/db_structure.png)

# Criação de novo banco de dados para a análise de dados

Em seguida, criamos um novo BD para conter dados agregados a todos os demais BDs. Isto, conforme os critérios de seleção vistos na tabela AnalisesCovid, descrita a seguir.

In [18]:
class AnaliseCovid(PacienteBase):
    """
    Tabela de análises Covid-19 FAPESP

    Complementa a tabela Paciente com dados relevantes sobre atendimentos,
    exames e desfechos extraídos das tabelas ExamLabs e Desfechos para análise
    epidemiológica de casos associados ao COVID-19

    Atributos:
        ID_Paciente: Identificação única do paciente (32 caracteres alfanuméricos)
        ID_Atendimento: Identificação única do atendimento (32 caracteres alfanuméricos)
        IC_Sexo: Sexo do Paciente (F - Feminino; M - Masculino)
        AA_Nascimento: Ano de nascimento do Paciente (4 caracteres alfanuméricos)
        CD_Pais: País de residência do Paciente (2 caracteres alfanuméricos, BR ou XX)
        CD_UF: Unidade da Federação de residência do Paciente (enumeração de estados brasileiros)
        CD_Municipio: Município de residência do Paciente (nome do município ou None para anonimização)
        CD_CEPReduzido: CEP reduzido do paciente (None para anonimização)
        DT_Atendimento: Data de realização do atendimento no formato DD/MM/AAAA
        DT_Coleta: Data em que o material foi coletado para exame no formato DD/MM/AAAA
        DE_Exame: Descrição do exame realizado (ex: "Teste COVID-19", "Hemograma")
        DT_Desfecho: Data do desfecho do paciente (alta, óbito, etc.) no formato DD/MM/AAAA
        DE_Desfecho: Descrição do desfecho do paciente (ex: "Alta médica melhorado", "Óbito")
        DE_Resultado: Resultado do exame COVID-19 simplificado (P - Positivo, N - Negativo, None - Outro/Indeterminado)
        DE_Hospital: Identificação do hospital de origem dos dados (BPSP, Einstein, GrupoFleury, HC, HSL)
    """

    __tablename__: str = "AnalisesCovid"
    index: Mapped[int] = column(autoincrement=True, primary_key=True)

    # Aggregated data
    ID_Paciente: Mapped[str] = column(String(32))
    ID_Atendimento: Mapped[str | None] = column(String(32))
    DT_Coleta: Mapped[Date | None]
    DT_Atendimento: Mapped[Date | None]
    DT_Desfecho: Mapped[str | None]
    DE_Desfecho: Mapped[str | None]
    DE_Exame: Mapped[str | None]
    DE_Resultado: Mapped[str | None]

    # Added metadata
    DE_Classe: Mapped[str | None] = column(String(1))
    DE_Hospital: Mapped[str]


# Populate a new database D2, solely with the AnalisesCovid table
d2_engine = create_engine(f"sqlite:///{cwd}/../fapcov2103/D2.db")
SessionD2 = sessionmaker(bind=d2_engine)
session_d2 = SessionD2()

Base.metadata.drop_all(d2_engine)
AnaliseCovid.__table__.create(d2_engine)
session_d2.commit()

## População do BD D2 com dados dos demais BDs

Em seguida acessamos aos demais BDs um a um e criamos registros correspondentes no DB D2:

In [19]:
pbar = tqdm(hospitals)
for hospital in pbar:
    pbar.set_description(f"Populating D2 database with data from {hospital}")
    hospital_engine = create_engine(f"sqlite:///{cwd}/../fapcov2103/{hospital}.db")
    SessionHospital = sessionmaker(hospital_engine)
    session_hospital = SessionHospital()

    query = (
        session_hospital.query(
            Paciente,  # This gives us all Paciente columns (P.*)
            Desfecho.DT_Desfecho,
            Desfecho.DT_Atendimento,
            ExamLab.ID_Atendimento,
            ExamLab.DT_Coleta,
            Desfecho.DE_Desfecho,
            ExamLab.DE_Exame,
            ExamLab.DE_Resultado,
        )
        .join(ExamLab, Paciente.ID_Paciente == ExamLab.ID_Paciente)
        .join(
            Desfecho,
            (Paciente.ID_Paciente == Desfecho.ID_Paciente)
            & (ExamLab.ID_Atendimento == Desfecho.ID_Atendimento),
        )
        .filter(
            ExamLab.DE_Exame.ilike("%covid%")
            | ExamLab.DE_Exame.ilike("%corona%")
            | ExamLab.DE_Exame.ilike("%sars%")
        )
    )

    for (
        paciente,
        dt_desfecho,
        dt_atendimento,
        id_atendimento,
        dt_coleta,
        de_desfecho,
        de_exame,
        de_resultado,
    ) in query:
        exam = ExamLab(DE_Exame=de_exame, DE_Resultado=de_resultado)

        match exam.DE_resultNum:
            case -1000:
                classe = "P"
            case -1111:
                classe = "N"
            case _:
                classe = None

        session_d2.add(
            AnaliseCovid(
                ID_Paciente=paciente.ID_Paciente,
                ID_Atendimento=id_atendimento,
                IC_Sexo=paciente.IC_Sexo,
                AA_Nascimento=paciente.AA_Nascimento,
                CD_UF=paciente.CD_UF,
                CD_Pais=paciente.CD_Pais,
                CD_Municipio=paciente.CD_Municipio,
                CD_CEPReduzido=paciente.CD_CEPReduzido,
                DT_Atendimento=dt_atendimento,
                DT_Coleta=dt_coleta,
                DE_Exame=de_exame,
                DT_Desfecho=dt_desfecho,
                DE_Desfecho=de_desfecho,
                DE_Resultado=de_resultado,
                DE_Classe=classe,
                DE_Hospital=hospital,
            )
        )

    session_hospital.close()
    session_d2.commit()
df = pd.read_sql_table("AnalisesCovid", d2_engine)
session_d2.close()
print(df.head())

Populating D2 database with data from HSL: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:07<00:00, 13.47s/it]


   index       ID_Paciente                    ID_Atendimento   DT_Coleta  \
0      1  B8E5614AFF218E3A  FB331193F22D79C578C5C2926FB8F611  23/04/2020   
1      2  B8E5614AFF218E3A  FB331193F22D79C578C5C2926FB8F611  04/04/2020   
2      3  263ED3AE6BCBA455  63736B20EC514D1201B1091BF256C04A  05/10/2020   
3      4  A948293FBD96829F  C385FCF6D86F8453977BCE950A2BA52D  27/10/2020   
4      5  7734B38BDC43E2F9  7B0F89CE1685F4131AD876BFBB64A82A  07/05/2020   

  DT_Atendimento DT_Desfecho                 DE_Desfecho  \
0     12/03/2020  24/04/2020              Alta melhorado   
1     12/03/2020  24/04/2020              Alta melhorado   
2     05/10/2020  07/10/2020              Alta melhorado   
3     27/10/2020  27/10/2020         Alta Administrativa   
4     07/05/2020  07/05/2020  Alta do Pronto Atendimento   

                               DE_Exame              DE_Resultado DE_Classe  \
0                  Coronavirus Covid-19             NAO DETECTADO         N   
1                  Coron